In [1]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio,transformar_tipo_identificacion, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, agregar_fuente, agregar_observacion, cargar_dfpaciente
import os
from dotenv import load_dotenv


2025/06/29


In [2]:
# Cargar variables del archivo .env
load_dotenv()

True

In [3]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [4]:
ruta_fecha = generar_ruta_fecha() 

In [5]:
 #3. Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-bronze",
    ruta_archivo=f"laboratorios/{ruta_fecha}/laboratorio Citopat/citopat.csv",
    tipo_archivo="csv"
)
print(df_datos)  # Mostrar las primeras filas del DataFrame extraído

      PROTOCOLO      CEDULA                      NOMBRE  EDAD SEXO  TELEFONO  \
0         12023    27451216     ESPERANZA NOGUERA PABON    54    F       NaN   
1         22023    37050039        MARIA FLORALBA ERAZO    67    F       NaN   
2         32023    30730297       AMPARO ROSERO MIDEROS    60    F       NaN   
3         42023  1108338133            JUAN DAVID MUÑOZ    10    M       0.0   
4         52023    27433207         ALBA LIGIA MARTINEZ    57    F       NaN   
...         ...         ...                         ...   ...  ...       ...   
6264   62652023    27455629  TERESA ORFELINA ARCINIEGAS    58    F       0.0   
6265   62662023    12957326     ALBERTO SIGIFREDO ERAZO    72    F       0.0   
6266   62672023    12970542        ALBERTO LASSO TORRES    63    M       0.0   
6267   62682023    27308551             ANA LUCIA BACCA    52   F.       0.0   
6268   62692023  1004236380          ANDRES FELIPE DIAZ    21    M       0.0   

           MEDICO REMITENTE DIAGNOSTICO

In [6]:
def transformar_df_lab_Citopat(df_datos):
    """
    Transforma un DataFrame del laboratorio Jimenes a la estructura de id_infolab
    
    Parámetros:
    - df_datos: DataFrame con los campos originales del laboratorio
    
    Retorna:
    - DataFrame transformado con la estructura deseada
    """
    
    # Crear copia para no modificar el original
    #df_transformado = df_datos.copy()
    
    # Mapeo de campos
    mapeo_campos = {
        'RECIBIDO': 'fecha_ingreso',
        'ENTREGADO': 'fecha_informe',
        'ENTIDAD': 'entidad',
        'MEDICO REMITENTE': 'medico_remitente',
        'MATERIAL ENVIADO': 'muestra_remitida',
        'DESCRIPCION MACROSCOPICA': 'descripcion_macroscopica',
        'DESCRIPCION MICROSCOPICA': 'descripcion_microscopica',
        'DIAGNOSTICO': 'diagnostico',
        'PATOLOGO': 'medico_ordenante',
        'CEDULA': 'NumeroIdentificacion',
        'NOMBRE': 'Nombre del Paciente',
        'CIE-10': 'cie10',
        'EDAD': 'Edad',
        'SEXO': 'sexo',
        'DIAGNOSTICO CLINICO':'diagnostico_clinico',
        'TELEFONO': 'telefono',
       
    }
    
    # Renombrar columnas
    df_transformado = df_datos.rename(columns=mapeo_campos)
    
    return df_transformado

In [7]:
df = transformar_df_lab_Citopat(df_datos)
df = transformar_tipo_identificacion(df)
df = agregar_fuente(df, "056")
df = agregar_observacion(df)
print(df.columns)  # Mostrar las columnas del DataFrame transformado

Index(['PROTOCOLO', 'NumeroIdentificacion', 'Nombre del Paciente', 'Edad',
       'sexo', 'telefono', 'medico_remitente', 'diagnostico_clinico',
       'muestra_remitida', 'fecha_ingreso', 'fecha_informe',
       'descripcion_macroscopica', 'descripcion_microscopica', 'diagnostico',
       'medico_ordenante', 'CODIGO DE COBRO', 'cie10', 'entidad',
       'TipoIdentificacion', 'id_paciente', 'fuente', 'comentarios',
       'observacion'],
      dtype='object')


In [8]:
connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}
insert_into_info_lab(df, connection_params)

Se insertaron 6269 filas en la tabla 'infolab'.


In [9]:
df_paciente = df[['NumeroIdentificacion', 'Nombre del Paciente', 'sexo', 'Edad', 'telefono', 'id_paciente','TipoIdentificacion']].copy()
df_paciente = df_paciente.drop_duplicates(subset=['id_paciente'], keep='first')
df_paciente['NumeroIdentificacion'] = df_paciente['NumeroIdentificacion'].astype(str).str.replace('.0', '', regex=False)
print(df_paciente.head())  # Mostrar las columnas del DataFrame transformado

  NumeroIdentificacion      Nombre del Paciente sexo  Edad  telefono  \
0             27451216  ESPERANZA NOGUERA PABON    F    54       NaN   
1             37050039     MARIA FLORALBA ERAZO    F    67       NaN   
2             30730297    AMPARO ROSERO MIDEROS    F    60       NaN   
3           1108338133         JUAN DAVID MUÑOZ    M    10       0.0   
4             27433207      ALBA LIGIA MARTINEZ    F    57       NaN   

    id_paciente TipoIdentificacion  
0    CC27451216                 CC  
1    CC37050039                 CC  
2    CC30730297                 CC  
3  TI1108338133                 TI  
4    CC27433207                 CC  


In [10]:
def renombrar_campos_paciente(df_paciente):
    """
    Renombra los campos del DataFrame df_paciente a los nombres especificados.

    Parámetros:
    - df_paciente: DataFrame con los campos originales.

    Retorna:
    - DataFrame con los campos renombrados.
    """
    mapeo_campos = {
        'Nombre del Paciente': 'nombre1',  # Asumiendo que este campo contiene el primer nombre
        'NumeroIdentificacion': 'identificacion',
        'sexo': 'sexo',
        'Edad': 'edad',
        'TipoIdentificacion': 'tipo_identificacion',    
        'id_paciente': 'id_paciente',
        # Agregar campos adicionales si están disponibles en el DataFrame original
    }
    
    # Renombrar columnas
    df_paciente_renombrado = df_paciente.rename(columns=mapeo_campos)
    
    # Agregar columnas faltantes con valores nulos
    columnas_faltantes = [
        'nombre2', 'nombre3', 'apellido1', 'apellido2',
        'fecha_nacimiento', 'municipio_residencia', 'codigo_residencia',
        'barrio_residencia', 'direccion'
    ]
    for columna in columnas_faltantes:
        if columna not in df_paciente_renombrado.columns:
            df_paciente_renombrado[columna] = None
    
    return df_paciente_renombrado

In [11]:
df_paciente_renombrado = renombrar_campos_paciente(df_paciente)
print(df_paciente_renombrado.head())

  identificacion                  nombre1 sexo  edad  telefono   id_paciente  \
0       27451216  ESPERANZA NOGUERA PABON    F    54       NaN    CC27451216   
1       37050039     MARIA FLORALBA ERAZO    F    67       NaN    CC37050039   
2       30730297    AMPARO ROSERO MIDEROS    F    60       NaN    CC30730297   
3     1108338133         JUAN DAVID MUÑOZ    M    10       0.0  TI1108338133   
4       27433207      ALBA LIGIA MARTINEZ    F    57       NaN    CC27433207   

  tipo_identificacion nombre2 nombre3 apellido1 apellido2 fecha_nacimiento  \
0                  CC    None    None      None      None             None   
1                  CC    None    None      None      None             None   
2                  CC    None    None      None      None             None   
3                  TI    None    None      None      None             None   
4                  CC    None    None      None      None             None   

  municipio_residencia codigo_residencia barrio_re

In [12]:
cargar_dfpaciente(df_paciente_renombrado,'paciente', connection_params)

Insertados 1 registros nuevos en paciente.


(True, 1)